Decision Trees는 SVM처럼 분류와 회귀에서 다재다능한 ML algorithm 이다. 복잡한 dataset에서 힘을 발위한다. 예를들어 2장에서 다뤄봤다. Decision Tree는 또한 7장에서 배울 Random Forests의 기본 구성요소이기도 하다. 이번 장은 어떻게 학습되고 보여지고 예측을 하는지 본다. 그 다음 Scikit-Learn에서 사용한 CART 훈련 algorithm을 살펴보고 정규화하고 회귀 작업에 사용하는 방법에 대해 알아본다. 마지막으로는 제한 사항에 대해서 설명한다.


# Training and Visualizing a Decision Tree



In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [3]:
from sklearn.tree import export_graphviz

export_graphviz(tree_clf, out_file = "iris_tree.dot",
               feature_names = iris.feature_names[2:],
               class_names = iris.target_names, rounded=True, filled=True)


**$ dot -Tpng iris_tree.dot -o iris_tree.png**


명렁어 쓰면 png 파일로바뀜

mac의 경우

$ brew install graphviz

먼저 설치

![iris_tree](./iris_tree.png)



# Making Predictions

위의 사진을 보면 root node에서 petal length가 2.45보다 작거나 같으면 왼쪽 자식 node로 간다. (depth 1, left) 즉 주황색은 leaf node 라 한다. (자식 노드가 없음) 
이때 class는 setosa 이다. 

node의 gini는 0으로 갈수록 잘 된거라 볼수 있다. 

밑의 식은 gini 계산을 나타냄.

![gini](../book_images/06/01.png)

pi, k 는 i번째 노드의 학습 인스턴스 중 클래스 k 인스턴스의 비율이다.

![boundaries](../book_images/06/02.png)

위의 그림은 Decision Tree의 의사 결정 단계를 보여준다. 



# Estimating Class Probabilities

Decision Tree 는 인스턴스가 특정 클래스 K에 속할 확률을 추정할수도 있다. 

먼저 트리의 리프 노드를 찾은다음 노드에서 클래스 k의 트레이닝 인스턴스 비율을 반환한다. 
예를들어 꽃잎이 길이 5cm , 너비 1.5cm인 꽃을 발견했을때 깊이 2의 왼쪽노드 이다. 위쪽에 사진을 봤을때 초록색 박스에 해당한다. setosa일 확률은 0/54 Versicolor일 확률은 49/54 Virginica일 확률은 5/54 이다. Versicolor가 가장 높은 확률이므로 출력한다. 

In [8]:
tree_clf.predict_proba([[5,1.5]])

array([[ 0.        ,  0.90740741,  0.09259259]])

In [9]:
tree_clf.predict([[5,1.5]])

array([1])

예상과 같게 결과가 나왔다. 

# The CART Training Algorithm

Scikit-Learn 은 Classification and Regression Tree (CART) 를 사용해 Decision Tree를 학습한다. 이 아이디어는 간단한데 단일 특징 k와 임계값 Tk (예 꽃잎 길이 <= 2.45cm)를 사용해 두개의 subsets을 사용해 어떻게 k와 Tk를 선택하냐면  가장 purest한 subsets을찾는다.

![CART](../book_images/06/03.png)

다음은 cost function을 나타냄.

훈련 세트를 두개로 나누면 동일한 logic을 사용해 분할 한다음 하위집합들을 재귀적으로 분할 한다. 최대깊이에 도달하거나 분할 할수 없을 경우 재귀를 중단한다. 다음 hyperparameters들이 정지 조건을 제어한다. 

(min_samples_split, min_samples_leaf, min_weight_fraction_leaf, and max_leaf_nodes).

불행히 최적의 트리를 찾는 것은 NP-Complete 문제로 알려져 있다. 
O(exp(m)) 시간이 필요하므로 상당히 작은 훈련 세트라도 다루기 어렵다. 

# Computational Complexity

예측을 하려면 Decision Tree 의 루트 노드에서 리프노드로 이동한다. 대략적으로 균형을 이뤄서 탐색시간은 O(log밑2(m)) 개의 노드를 통과해야한다. 각 노드는 하나의 값을 확인만 하면 되므로 예측 복잡성은 O(log밑2(m))에 불과하다. 따라서 대규모 교육 세트를 다룰때에도 예측은 빠르다. 

하지만 학습 알고리즘은 각 node에서 비교를 거치는데 이것은 O (n × m log (m)) 복잡성을 가진다. 작은 훈련 세트의 경우 train 속도는 빠르지만 대규모의 데이터면 상당히 늦어진다.

# Gini Impurity or Entropy?

기본으로 Gini impurity measure가 사용되지만 hyperparameter의 "entropy"를 설정하여 entropy impurity measure를 선택할 수 있다. entropy의 개념은 열역학에서 유래 했는데 분자가 잘 정렬되어있을때 entropy는 제로에 접근한다. 여기서는 모든 메시지가 동일할때 entropy가 0이다. 집합의 entropy는 하나의 클래스만의 인스턴스가 포함되어있을 때 0이다.

![entropy](../book_images/06/04.png)

위의 식은 i번째 노드의 entropy의 정의를 보여준다. 

그래서 Gini impurity or entropy를 사용해야하냐?  
대부분의 경우 큰 차이가 없다. 비슷한 tree로 만든다. Gini impurity는 약간 속도가 빨라져서 기본값으로 사용한다. 하지만 서로 다른경우 Gini impurity는 가장 빈번한 클래스를 분리하는 경향이 있고 entropy는 균형적으로 tree를 만드는 경향이 있다.

# Regularization Hyperparameters

Decision Trees는 교육데이터에 대해 가정하지않는다. ( 선형모델과는 달리 데이터가 선형이라고 가정함. ) 제한되지 않을채로 둔다면 학습 데이터에 너무 잘 맞춰질수 있다. 모델은 매개변수가 없기 때문에 매개 변수 수가 훈련을 하기전에 결정되지 않았기 때문에가 아닌 모델을 nonparametric model 이라고 부른다. 따라서 모델 구조는 데이터에 따라 자유롭게 만들어질수있다. 대조적으로 선형 모델과 같이 매개 변수 모델은 미리 결정된 수의 매개변수를 가지므로 자유도가 제한되어서 오버피팅의 위함을 줄인다. ( 하지만 언더피팅의 위험이 있다. )

교육 데이터의 오버피팅을 피하려면 Decision Tree의 교육중 자유도를 제한해야한다. 이를 정규화라고 한다. 정규화 hyperparameters는 사용된 알고리즘에 따라 다르지만 일반적으로 Decision Tree의 깊이를 제한 할 수 있다. Scikit-Learn에서는 max-depth hyperparameter에 의해 제어된다. 

![entropy](../book_images/06/05.png)

위 사진은 훈련된 2개의 Decision Tree를 보여준다. 왼쪽은 기본 매개변수이고 오른쪽은 min_samples_leaf = 4로 훈련된다. 왼쪽 모델이 오버 피팅이고 잘 일반화 될 것이다.

# Regression

Decision Tree는 회귀 작업을 수행할 수도 있다. Scikit-Learn의 DecisionTreeRegressor 클래스를 사용해 회귀 트리를 만들고 max_depth = 2인 노이지가 있는 데이터셋에서 학습을 해라.

In [10]:
from sklearn.tree import DecisionTreeRegressor 

tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [11]:
from sklearn.tree import export_graphviz

export_graphviz(tree_reg, out_file = "reg.dot",
               feature_names = iris.feature_names[2:],
               class_names = iris.target_names, rounded=True, filled=True)

![reg](./reg.png)



책과는 결과가 다르게 나왔다. 

![entropy](../book_images/06/06.png)


이 트리는 이전에 본 분류 트리와 비슷하다. 차이점은 각 노드의 클래스를 예측하는 대신 값을 예측한다.
예를들어 x1 = 0.6인 새 인스턴스에 대해 예측을하고 싶다고 가정하면 루트에서 시작해 트리를 탐색하면 0.1106을 예측하는 리프 노드에 도달한다. 이 예측은 리프 노드와 연관된 110 개의 학습 인스턴스의 평균 목표 값이다. 이 예측은 110 개의 인스턴스에 대해 MSE가 0.0151이 되도록 한다.

이 모델의 예측은 밑의 그림에 왼쪽에 표시되는데 max_depth = 3으로 설정하면 오른쪽에 표시된 값을 얻을수 있다.

![entropy](../book_images/06/07.png)

CART알고리즘은 impurity를 최소화하는 방식으로 training set을 분활 하는 대신 MSE를 최소화 하는 방식으로 training set를 분할 하려고 하는 점을 제외하고 이전과 같다. 밑의 식은 cost 함수를 보여준다.

![entropy](../book_images/06/08.png)

분류 작업과 마찬가지로 Decision Tree는 회귀 작업을 처리 할때 오버피팅이 되는 경우가 있다. 정규화가 없으면 밑의 그림의 왼쪽에 예측값이 표시된다. 오버피팅이된것을 볼수 있다. min_samples_leaf = 10 으로 설정하면 밑의 그림의 오른쪽에 모델로 훨씬 좋은 결과를 얻었다.

![entropy](../book_images/06/09.png)

# Instability

다행히 Decision Tree는 이해하기 쉽고 해석하기 쉽고 사용하기 쉽고 다양하고 강력하당. 하지만 몇가지 제한이 있는데 먼저 직교로 경계를 친다. 모든분할이 축에 수직으로 이루어져 있다. 예를 들어 밑의 그림은 단순히 선형 분리 데이터세트를 보여주는데 왼쪽은 Decision Tree로 쉽게 분리 하지만 오른쪽은 확실히 복잡하다. 오른쪽 모델은 일반화가 잘 되진 않을 것이다. 이 문제를 제한하는 방법은 PCA를 사용한다. (8장) 

![entropy](../book_images/06/10.png)

일반적으로 Decision Tree의 문제점은 데이터의 작은 변화에 민감하다. 예를들어 (트레이닝 세트에서 꽃잎이 길이 4.8 cm 너비가 1.8cm ) 가장 넓은 데이터를 제거하고 학습하면 밑의 그림과 같은 모델을 얻을수 있다. 이는 전에 했던 Decision Tree의 결과와 매우다르다.  실제로 Scikit-Learn에 의해 사용 된 훈련 알고리즘이 확률 적이라서 동일한 훈련데이터에서도 다른 모델을 얻을수 있다. 

![entropy](../book_images/06/11.png)

Random Forest는 다음장에서 다르는데 많은 tree에 대한 예측을 평균해 불안정을 제한 할 수 있다.